<center><b>  <span style="font-size:24px;">FINA 4335 -Final Project-<span style="color:red;">Part II <span></span></b>
<center><span style="font-size:22px;"><b>Due date: 12/11/2024 11:59 PM ET</b></span></center>
            
##### First name and Last name (team member 1): Andre Araujo-Kerry
##### First name and Last name (team member 2): Dominic Imondi
##### First name and Last name (team member 3): Mikaela Moskowitz

## Instructions
1. Please rename the notebook. It should have the following format: **Final Project_Part II_Lastname1_Lastname2** if there are 2 members in the team.
1. Use this **template** and you are free to add more code/markdown cells. Change the markdown cells if necessary. Well format this notebook file.
1. After you answer all questions:
    1. Restart your kernel and run all cells
    1. Save your notebook and export your notebook as HTML
    1. Submit the **notebook** and **HTML** to Canvas, under Final Group Project.

## Some Grading Guidelines
1. All code cells (excluding evaluation cells) should be **executable**. 
    - Non-executable code cells will receive up to **50%** of grade deduction. To receive more partial credit for a non-executable code cell, ensure it includes sufficient comments.
1.  <span style="color: blue;">I grade on both **correctness** and **effort**</span>
    - **Suppose you don’t know how to generate a specific variable, then you <span style="color: red;">may use random values (with somewhat reasonable range) to replace the variable</span>. You will lose point for that variable but will get most credit for subsequent questions.**
1. Write efficient and clean code
    -  No repetitive code (An example of repetitive code: import the same csv file twice).
    - Informative variable names (if not specified in the project description).
    - Less than 2 uncommented `print`, `.describe()`, or similar quality check in every code cell or every 5 lines of code.
    - Every 2 occurrence of inefficient or inclean code will receive **1 point deduction**
1. Preserve as many observations as possible
    - Losing observations is considered partially incorrect code
    - Do not fill NAs
1. Include sufficient comments (at least 1 comment every 3 lines of code)
      - Failure to include sufficient comments will result in a **5 points** grade deduction
1. Format the markdown file such as including headings and brief question description
1. Use the methods and formulas **covered in the lecture notebooks**. Using formulas inconsistent to the lecture notebooks receives 20% of grade penalty
1. Reasonable code execution time
   - The code execution time for this Part should be less than 20 minutes, otherwise **deduct 1 point for every 5 minutes more code execuation time**
1. **10% of grade penalty per day late, receive 0% if the project is submitted after Dec 13 11:59 PM ET.**

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import scipy.stats as stats 
import statsmodels.api as sm
import pandas_datareader.data as web
import matplotlib.pyplot as plt
%matplotlib inline
import time
%config InlineBackend.figure_format = 'retina'
# Measure execution time
start_time_part2 = time.time()  # Start time

## Data preparation (5 points)
**Explanation of variables in 'weekly_rets.csv'**

- **DATE**: This variable represents the last trading day of each week.
- **PERMNO**: This is a unique identifier assigned to each security.
- **ret**: the stock's weekly return, expressed as a decimal.
  
**Explanation of variables in 'monthly_rets_funda.csv'**

- **DATE**: This variable represents the last trading day of each month.
- **PERMNO**: This is a unique identifier assigned to each security.
- **PRC**: reflects the **unadjusted** close price of a stock on the last trading day of the month. This value is the raw closing price without adjustments for corporate actions such as stock splits, dividends, or other modifications.
- **RET**: the stock's monthly return, expressed as a decimal. 
- **SHROUT**: The **shares outstanding** (SHROUT) variable represents the number of shares (in thousands) of a company’s stock that are currently held by all shareholders.
- **revenues**:  This variable represents the total revenues for a specific fiscal year. The variable is already 'lagged'.

In [2]:
# Write your code in the provided code cell; you can add more cells (same applies below, no further repetition).

#read weekly_rets
weekly_rets = pd.read_csv('weekly_rets.csv')
#parse DATE column to datetime format
weekly_rets['DATE'] = pd.to_datetime(weekly_rets['DATE'],format='%Y%m%d')
#sort values by PERMNO and DATE
weekly_rets.sort_values(['PERMNO','DATE'], inplace=True)

#read monthly_rets_funda
monthly_rets = pd.read_csv('monthly_rets_funda.csv')
#parse DATE column to datetime format
monthly_rets['DATE'] = pd.to_datetime(monthly_rets['DATE'],format='%Y%m%d')
#sort values by PERMNO and DATE
monthly_rets.sort_values(['PERMNO','DATE'], inplace=True)

In [3]:
# Evaluation cell (used for grading-I don't have a TA)
# you don't need to worry at all if you think certain valuation cell is wrong (same applies below, no further repetition)
print(weekly_rets.head())
print(monthly_rets.head())

        DATE  PERMNO       ret
0 2008-01-04   10001 -0.028264
1 2008-01-11   10001 -0.013150
2 2008-01-18   10001 -0.004682
3 2008-01-25   10001  0.002904
4 2008-02-01   10001  0.026923
        DATE  PERMNO        PRC       RET    SHROUT  revenues
0 2008-01-31   10001  13.986000 -0.006076  2872.125    84.278
1 2008-02-29   10001   9.490400  0.021682  4340.655    84.278
2 2008-03-31   10001   8.956035 -0.052463  4342.653    84.278
3 2008-04-30   10001   8.591400 -0.036661  4342.653    84.278
4 2008-05-30   10001  10.889100  0.271356  4342.653    84.278


## Constructing factors (33 points; 15+8+5+5)

### Create signals  (3*5=15 points)

#### price_to_sales

In [73]:
# revenue is already 'lagged'

#create a copy of monthly_rets
ptc_monthly = monthly_rets.copy()

#lag the stock price and shares outstanding
#revenue is already lagged so leave as-is
ptc_monthly['PRC_lagged'] = ptc_monthly.groupby('PERMNO')['PRC'].shift(1)
ptc_monthly['SHROUT_lagged'] = ptc_monthly.groupby('PERMNO')['SHROUT'].shift(1)

#get sales figure (revenues / shares outstanding in month T - 1)
ptc_monthly['Sales'] = ptc_monthly['revenues'] / ptc_monthly['SHROUT_lagged']
#calculate price-to-sales (price in month T - 1 / sales)
ptc_monthly['P/S'] = ptc_monthly['PRC_lagged'] / ptc_monthly['Sales']

#drop N/A P/S values
#TODO (monthly_rets left outer join ptc_monthly) to rejoin the tables and preserve all the observations
ptc_monthly = ptc_monthly.dropna(subset=['P/S'])

#add a price-to-sales bucket column
ptc_monthly['ptc_bucket'] = (
    #group by date to group individual months, and apply qcut on price-to-sales with 5 buckets
    ptc_monthly.groupby('DATE', group_keys=False)['P/S']
    .apply(pd.qcut, q=5, labels=np.arange(5) + 1)
)

#create the signal
#1 for bottom quintile (long), -1 for top quintile (short), else np.nan
ptc_monthly['price_to_sales'] = np.where(ptc_monthly['ptc_bucket'] == 1, 1, 
                                np.where(ptc_monthly['ptc_bucket'] == 5, -1, np.nan))

In [75]:
# some quality check such as .describe() and print head of key variables
print(ptc_monthly.head())
print(ptc_monthly.describe())

        DATE  PERMNO        PRC       RET    SHROUT  revenues  PRC_lagged  \
1 2008-02-29   10001   9.490400  0.021682  4340.655    84.278   13.986000   
2 2008-03-31   10001   8.956035 -0.052463  4342.653    84.278    9.490400   
3 2008-04-30   10001   8.591400 -0.036661  4342.653    84.278    8.956035   
4 2008-05-30   10001  10.889100  0.271356  4342.653    84.278    8.591400   
5 2008-06-30   10001  10.739250 -0.010082  4344.651    84.278   10.889100   

   SHROUT_lagged     Sales         P/S ptc_bucket  price_to_sales  
1       2872.125  0.029343  476.631390          1             1.0  
2       4340.655  0.019416  488.793667          2             NaN  
3       4342.653  0.019407  461.484044          1             1.0  
4       4342.653  0.019407  442.695235          1             1.0  
5       4342.653  0.019407  561.090472          2             NaN  
                                DATE         PERMNO            PRC  \
count                         500211  500211.000000  497848

#### ret_2to12m

In [34]:
# Each signal is only worth 3 points. Suppose you don’t know how to generate a specific signal, then you may use random values 
# (with somewhat reasonable range) to replace the signal. You will lose 3 points but will get most credit for subsequent questions.

#create a copy of monthly rets
momentum_monthly = monthly_rets.copy()

#lag the returns by 2 months (to get return at month T - 2)
momentum_monthly['RET_lagged'] = momentum_monthly.groupby('PERMNO')['RET'].shift(2)

#find the cumulative return for each 11 month period
momentum_monthly['cumulative_ret'] = (
    momentum_monthly.groupby('PERMNO')['RET_lagged'] #group by each asset
    .rolling(window=11, min_periods=11) #rolling with 11 periods, from month t - 12 to month t - 2
    .apply(lambda x: np.prod(1 + x) - 1, raw=False) #use np.prod to get cumulative returns
    .reset_index(level=0, drop=True) #reset indices
)

#drop N/A cumulative return values
#TODO (monthly_rets left outer join momentum_monthly) to rejoin the tables and preserve all the observations
momentum_monthly = momentum_monthly.dropna(subset=['cumulative_ret'])

#add a cumulative return bucket column
momentum_monthly['cumulative_ret_bucket'] = (
    #group by date to group individual months, and apply qcut on cumulative return with 5 buckets
    momentum_monthly.groupby('DATE', group_keys=False)['cumulative_ret']
    .apply(pd.qcut, q=5, labels=np.arange(5) + 1)
)

#create the signal
#-1 for bottom quintile (short), 1 for top quintile (long), else np.nan
momentum_monthly['ret_2to12m'] = np.where(momentum_monthly['cumulative_ret_bucket'] == 5, -1, 
                                 np.where(momentum_monthly['cumulative_ret_bucket'] == 1, 1, np.nan))

In [79]:
# some quality check such as .describe() and print head of key variables
print(momentum_monthly.head())
print(momentum_monthly.describe())

         DATE  PERMNO       PRC       RET    SHROUT  revenues  RET_lagged  \
12 2009-01-30   10001  8.493498  0.034106  4292.703    59.373   -0.129991   
13 2009-02-27   10001  8.931060  0.056166  4292.703    59.373    0.155833   
14 2009-03-31   10001  8.171820 -0.080456  4295.700    59.373    0.034106   
15 2009-04-30   10001  8.491500  0.044576  4295.700    59.373    0.056166   
16 2009-05-29   10001  8.471520  0.002938  4349.646    59.373   -0.080456   

    cumulative_ret cumulative_ret_bucket  ret_2to12m  
12       -0.202369                     5        -1.0  
13       -0.072436                     5        -1.0  
14       -0.061156                     5        -1.0  
15        0.046476                     5        -1.0  
16       -0.001099                     5        -1.0  
                                DATE         PERMNO            PRC  \
count                         455096  455096.000000  452984.000000   
mean   2016-05-31 14:35:08.917678848   59550.206614      40.207298 

#### ret_2w

In [78]:
# Each signal is only worth 3 points. Suppose you don’t know how to generate a specific signal, then you may use random values 
# (with somewhat reasonable range) to replace the signal. You will lose 3 points but will get most credit for subsequent questions.

#make a copy of monthly rets & add a month-year column for future merging
ret_2w_monthly = monthly_rets.copy()
ret_2w_monthly['month-year'] = ret_2w_monthly['DATE'].dt.to_period('M')

#add a month-year column to organize weekly data by month
weekly_rets['month-year'] = weekly_rets['DATE'].dt.to_period('M')

#for every month, cumulative return is the sum of the last 2 weeks
weekly_rets['cumulative_ret'] = (
    weekly_rets.groupby(['PERMNO', 'month-year'])['ret']
    .transform(lambda x: x.tail(2).sum())  # Sum the last 2 weeks for each group
)

#aggregate weekly data to get one monthly cumulative return
weekly_monthly_agg = weekly_rets.groupby(['PERMNO', 'month-year'])['cumulative_ret'].last().reset_index()

#merge weekly and monthly data on their month-year values
ret_2w_monthly = pd.merge(ret_2w_monthly, weekly_monthly_agg, on=['PERMNO', 'month-year'], how='left')
#lag the data by 1 year
ret_2w_monthly['cumulative_ret_lagged'] = ret_2w_monthly.groupby('PERMNO')['cumulative_ret'].shift(1)

#drop N/A cumulative return values
#TODO (monthly_rets left outer join ret_2w_monthly) to rejoin the tables and preserve all the observations
ret_2w_monthly = ret_2w_monthly.dropna(subset=['cumulative_ret_lagged'])

#add a cumulative return bucket column
ret_2w_monthly['cumulative_ret_bucket'] = (
    #group by date to group individual months, and apply qcut on cumulative return with 5 buckets
    ret_2w_monthly.groupby('DATE', group_keys=False)['cumulative_ret_lagged']
    .apply(pd.qcut, q=5, labels=np.arange(5) + 1)
)

#create the signal
#1 for bottom quintile (long), -1 for top quintile (short), else np.nan
ret_2w_monthly['ret_2w'] = np.where(ret_2w_monthly['cumulative_ret_bucket'] == 1, 1, 
                                   np.where(ret_2w_monthly['cumulative_ret_bucket'] == 5, -1, np.nan))

In [81]:
# some quality check such as .describe() and print head of key variables
print(ret_2w_monthly.head())
print(ret_2w_monthly.describe())

        DATE  PERMNO        PRC       RET    SHROUT  revenues month-year  \
1 2008-02-29   10001   9.490400  0.021682  4340.655    84.278    2008-02   
2 2008-03-31   10001   8.956035 -0.052463  4342.653    84.278    2008-03   
3 2008-04-30   10001   8.591400 -0.036661  4342.653    84.278    2008-04   
4 2008-05-30   10001  10.889100  0.271356  4342.653    84.278    2008-05   
5 2008-06-30   10001  10.739250 -0.010082  4344.651    84.278    2008-06   

   cumulative_ret  cumulative_ret_lagged cumulative_ret_bucket  ret_2w  
1       -0.020056              -0.001779                     4     NaN  
2       -0.020564              -0.020056                     3     NaN  
3       -0.007235              -0.020564                     2     NaN  
4        0.126358              -0.007235                     2     NaN  
5        0.000727               0.126358                     5    -1.0  
                                DATE         PERMNO            PRC  \
count                         52159

#### mcap_quarter

In [ ]:
# Each signal is only worth 3 points. Suppose you don’t know how to generate a specific signal, then you may use random values 
# (with somewhat reasonable range) to replace the signal. You will lose 3 points but will get most credit for subsequent questions.


In [ ]:
# some quality check such as .describe() and print head of key variables


#### beta

In [ ]:
# Each signal is only worth 3 points. Suppose you don’t know how to generate a specific signal, then you may use random values 
# (with somewhat reasonable range) to replace the signal. You will lose 3 points but will get most credit for subsequent questions.


In [ ]:
# some quality check such as .describe() and print head of key variables


In [ ]:
#############################################################################################################################
# Evaluation cell (used for grading-I don't have a TA)
#############################################################################################################################
some_vars = ['PERMNO','RET']+['price_to_sales', 'ret_2to12m', 'ret_2w', 'mcap_quarter', 'beta']
print(mrets_with_signals[some_vars].head())
mrets_with_signals[some_vars].describe()

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
mrets_with_signals.head()

### Form time-series portfolios (8 points) 

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(value_df.head())
value_df.describe()

In [ ]:
# Evaluation cell (do not change; used for grading)
print(momentum_df.head())
momentum_df.describe()

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(reversal_df.head())
reversal_df.describe()

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(size_df.head())
size_df.describe()

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(beta_df.head())
beta_df.describe()

### plot (5 points)

### Save the 5 long-short factor returns in a DataFrame called `factors_df`. (Do not drop any missing values). (5 points)

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(factors_df.head())
factors_df.describe()

## Evaluate the factors you created (14 points; 4+4+6)

### Estimate Fama-French 5-factor model for the 5 factor portfolios (in `factors_df`). Report the R squares, regression coefficients, and their p values in a single DataFrame called `ff5_factors_reg`. 

### Create `factors_sumstat`. 

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(ff5_factors_reg)
print(factors_sumstat)

### Summarize at least three takeaways related to investments based on your results in Part II so far. Cite credible sources that help explain and support your findings. (6 points)

## Factor combination strategy (‘Mission Impossible’) (8 points)
-  You can get partial credit if you have reasonable code.

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
impossible_df

In [ ]:
end_time_part2 = time.time()  # End time
execution_time_part2 = end_time_part2 - start_time_part2
# Calculate the number of full minutes
minutes = int(execution_time_part2 // 60)
# Calculate the remaining seconds
seconds = int(execution_time_part2 % 60)  
print(f"Execution time for Part II of the project:{minutes} minutes and {seconds} seconds")